In [5]:
import sys, os, inspect

# add common to path
from pathlib import Path
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
common_path = str(Path(currentdir).parent.parent)
sys.path.append( common_path )

from common.nlp.vocab import Vocab
from common.nlp.data_loader import N21TextData
from common.nlp.converter import N21Converter

from dataset import SentimentDataset
from dataset import load_data
from common.ml.hparams import HParams

import numpy as np
import copy 
import time 
import tensorflow as tf
from tensorflow.python.ops import rnn
from tensorflow.contrib.layers.python.layers import linear

from common.ml.tf.deploy import freeze_graph

print( "Tensorflow Version : ", tf.__version__)

Tensorflow Version :  1.10.0


In [6]:
class SentimentAnalysis():
    def __init__(self, hps, mode="train"):
        self.hps = hps
        self.x = tf.placeholder(tf.int32, [None, hps.num_steps], name="pl_tokens")#None: batch_size를 모르기 때문에 알아서 하도록 함
        self.y = tf.placeholder(tf.int32, [None], name="pl_target")#placeholder: 다른 텐서를 할당하기 위해사용, 이름도 써주는게 좋음
        self.w = tf.placeholder(tf.float32, [None, hps.num_steps], name="pl_weight")
        self.keep_prob = tf.placeholder(tf.float32, [], name="pl_keep_prob")

        ### 4 blocks ###
        # 1) embedding
        # 2) dropout on input embedding
        # 3) sentence encoding using rnn
        # 4) encoding to output classes
        # 5) loss calcaulation

        def _embedding(x):
            # character embedding 
            #print("-"*100)
            #print("Implement function '{}'".format(_embedding.__name__))
            #print("Keywords")
            #print("\t - tf.initializers.variance_scaling")
            #print("\t - tf.get_variable")
            #print("\t - tf.nn.embedding_lookup")
            #print("\t - tf.unstack")

            #print('Input : Tensor("model/pl_tokens:0", shape=(?, 128), dtype=int32)')

            
            #print("Return: a list of <tf.Tensor shape=(?, 50) dtype=float32>")
            #print("        len(a_list) = 128")
            shape = [hps.vocab_size,hps.emb_size]
            initializer = tf.initializers.variance_scaling(distribution="uniform",dtype=tf.float32)#가중치 벡터의 shape에 맞게 조정
            emb_mat = tf.get_variable('emb',shape,initializer=initializer,dtype=tf.float32)#입력한 파라미터에 맞는 데이터를 생성하거나 가져옴
            input_emb = tf.nn.embedding_lookup(emb_mat,x)#embedding tensor리스트에서 id를 검색함
            
            step_inputs = tf.unstack(input_emb,axis=1) # split input_emb -> num_steps, tf.unstack: 랭크를 1줄임
            return step_inputs

        def _sequence_dropout(step_inputs, keep_prob):
            # apply dropout to each input
            # input : a list of input tensor which shape is [None, input_dim]
            print("-"*100)
            with tf.name_scope('sequence_dropout') as scope:
                #print("Implement step_outputs")
                #print("Keywords")
                #print("\t - tf.nn.dropout")
                step_outputs=[]
                for t,input in enumerate(step_inputs):
                    step_outputs.append(tf.nn.dropout(input,keep_prob))



            #print("Return: a list of <tf.Tensor shape=(?, 50) dtype=float32>")
            return step_outputs

        def sequence_encoding_n21_rnn(step_inputs, cell_size, scope_name):
            # rnn based N21 encoding (GRU)
            #print("-"*100)
            #print("Implement function '{}'".format(sequence_encoding_n21_rnn.__name__))
            #print('Input : a list of <tf.Tensor shape=(?, 50), dtype=float32>')
            #print("Keywords")
            #print("\t - tf.contrib.rnn.GRUCell")
            #print("\t - tf.contrib.rnn.static_rnn")

            #print("Return: a list of <tf.Tensor, shape=(?, 100)>")
            step_inputs=list(reversed(step_inputs))#뒤 쪽에 있는 패딩심볼의 악영향을 줄이기 위해 뒤에서 부터 인코딩
            f_rnn_cell=tf.contrib.rnn.GRUCell(cell_size,reuse=None)
            _inputs=tf.stack(step_inputs,axis=1)
            step_outputs, final_state=tf.contrib.rnn.static_rnn(f_rnn_cell,
                                                               step_inputs,
                                                               dtype=tf.float32,
                                                               scope=scope_name)#각 step별 output과 마지막 히든레이어를 결과로
            out = step_outputs[-1]            
            return out

        def _to_class(input, num_class):
            #print("-"*100)
            #print("Implement function '{}'".format(_to_class.__name__))
            #print('Input : tf.Tensor, shape=(?, 100), dtype=float32')
            #print("Keywords")
            #print("\t - tensorflow.contrib.layers.python.layers.linear")

            #print("Return: tf.Tensor, shape=(?, 4), dtype=float32")
            out = linear(input,num_class,scope="Rnn2Sentiment") # should be implemented
            return out

        def _loss(out, ref):
            # out : [batch_size, num_class] float - unscaled logits
            # ref : [batch_size] integer
            # calculate loss function using cross-entropy
            #print("-"*100)
            #print('Input out: tf.Tensor, shape=(?, 4)')
            #print('Input ref: tf.Tensor("model/pl_target:0", shape=(?,), dtype=int32)')

            #print("Keywords")
            #print("\t - tf.nn.sparse_softmax_cross_entropy_with_logits")
            #print("\t - tf.reduce_mean")
            
            #print("Return: tf.Tensor, shape=(), dtype=float32")
            batch_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=out,labels=ref,name="sentiment_loss")
            loss =  tf.reduce_mean(batch_loss)# [batch_size,emb_dim=30]
            return loss
        
        seq_length    = tf.reduce_sum(self.w, 1) # [batch_size]

        step_inputs   = _embedding(self.x) # [batch_size,emb_dim=50]
        step_inputs   = _sequence_dropout(step_inputs, self.keep_prob)
        sent_encoding = sequence_encoding_n21_rnn(step_inputs, hps.enc_dim, scope_name="encoder")#sentence encoding
        out           = _to_class(sent_encoding, hps.num_target_class)
        loss          = _loss(out, self.y) 

        if loss is None: 
            print("All functions should be implemented!")
            import sys; sys.exit()


        out_probs     = tf.nn.softmax(out, name="out_probs")
        out_pred      = tf.argmax(out_probs, 1, name="out_pred")

        self.loss      = loss
        self.out_probs = out_probs#[batch_size, num_class]
        self.out_pred  = out_pred#[batch_size]

        self.global_step = tf.get_variable("global_step", [], tf.int32, initializer=tf.zeros_initializer, trainable=False)

        if mode == "train":#backpropagation
            optimizer       = tf.train.AdamOptimizer(hps.learning_rate)#자연어에서는 주로 adam사용
            self.train_op   = optimizer.minimize(self.loss, global_step=self.global_step)
        else:
            self.train_op = tf.no_op()


    @staticmethod
    def get_default_hparams():
        return HParams(
            learning_rate     = 0.001,
            keep_prob         = 0.5,#문제의 손상율
        )

In [7]:
def train(train_id_data, num_vocabs, num_taget_class):
    #
    # train sentiment analysis using given train_id_data
    #
    max_epoch = 3000
    model_dir = "./trained_models"
    hps = SentimentAnalysis.get_default_hparams()
    hps.update(
                    batch_size= 100,
                    num_steps = 128,#몇 개의 char가 들어오나? 즉 input
                    emb_size  = 50,#조금 작은 값임
                    enc_dim   = 100,#마지막 시퀀스 인코더의 dimension
                    vocab_size=num_vocabs,
                    num_target_class=num_taget_class
               )

    with tf.variable_scope("model"):#해당 스코프안에서 정의되는 변수는 해당범위에만 한정되어짐
        model = SentimentAnalysis(hps, "train")

    sv = tf.train.Supervisor(is_chief=True,#프로그래밍 패턴중 하나, sv안에 세션을 담으면 알아서 다 해준다고 함
                             logdir=model_dir,
                             summary_op=None,  
                             global_step=model.global_step)

    # tf assign compatible operators for gpu and cpu 
    tf_config = tf.ConfigProto(allow_soft_placement=True)#cpu, gpu가 문제없이 돌아갈 수 있도록 해주는 역할

    with sv.managed_session(config=tf_config) as sess:
        local_step       = 0
        prev_global_step = sess.run(model.global_step)

        train_data_set = SentimentDataset(train_id_data, hps.batch_size, hps.num_steps)
        losses = []
        while not sv.should_stop():#미니배치로 실행
            fetches = [model.global_step, model.loss, model.train_op]
            a_batch_data = next( train_data_set.iterator )
            y, x, w = a_batch_data
            fetched = sess.run(fetches, {
                                            model.x: x, 
                                            model.y: y, 
                                            model.w: w,

                                            model.keep_prob: hps.keep_prob,
                                        }
                              )

            local_step += 1

            _global_step = fetched[0]
            _loss        = fetched[1]
            losses.append( _loss )
            if local_step < 10 or local_step % 10 == 0:
                epoch = train_data_set.get_epoch_num()
                print("Epoch = {:3d} Step = {:7d} loss = {:5.3f}".format(epoch, _global_step, np.mean(losses)) )
                _loss = []                
                if epoch >= max_epoch : break 

        print("Training is done.")
    sv.stop()

    # model.out_pred, model.out_probs
    freeze_graph(model_dir, "model/out_pred,model/out_probs", "frozen_graph.tf.pb") ## freeze graph with params to probobuf format


In [8]:
from tensorflow.core.framework import graph_pb2
def predict(token_vocab, target_vocab, sent):
    #os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # force to use cpu only (prediction)
    os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
    os.environ['CUDA_VISIBLE_DEVICES']='0'# force to use gpu only (prediction)
    model_dir = "./trained_models"

    # prepare sentence converting
    # to make raw sentence to id data easily
    in_sent = '{}\t{}'.format('___DUMMY_CLASS___', sent)
    pred_data     = N21TextData(in_sent, mode='sentence')
    pred_id_data  = N21Converter.convert(pred_data, target_vocab, token_vocab)
    pred_data_set = SentimentDataset(pred_id_data, 1, 128)

    #
    a_batch_data = next(pred_data_set.predict_iterator) # a result
    b_sentiment_id, b_token_ids, b_weight = a_batch_data

    # Restore graph
    # note that frozen_graph.tf.pb contains graph definition with parameter values in binary format
    _graph_fn =  os.path.join(model_dir, 'frozen_graph.tf.pb')
    with tf.gfile.GFile(_graph_fn, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)

    with tf.Session(graph=graph) as sess:
        # to check load graph
        #for n in tf.get_default_graph().as_graph_def().node: print(n.name)

        # make interface for input
        pl_token     = graph.get_tensor_by_name('import/model/pl_tokens:0')
        pl_keep_prob = graph.get_tensor_by_name('import/model/pl_keep_prob:0')

        # make interface for output
        out_pred  = graph.get_tensor_by_name('import/model/out_pred:0')
        out_probs = graph.get_tensor_by_name('import/model/out_probs:0')
        

        # predict sentence 
        b_best_pred_index, b_pred_probs = sess.run([out_pred, out_probs], feed_dict={
                                                                                        pl_token : b_token_ids,
                                                                                        pl_keep_prob : 1.0,
                                                                                    }
                                          )

        best_pred_index = b_best_pred_index[0]
        pred_probs = b_pred_probs[0]
        #결과 출력 루틴
        best_target_class = target_vocab.get_symbol(best_pred_index)
        print( pred_probs[best_pred_index] )
        best_prob  = int( pred_probs[best_pred_index] )
        print(best_target_class, best_prob)

In [9]:
train_id_data, token_vocab, target_vocab = load_data()#데이터 로드
num_vocabs       = token_vocab.get_num_tokens()#vocab로드
num_target_class = target_vocab.get_num_targets()#타겟(카테고리) 로드

#train(train_id_data, num_vocabs, num_target_class)#학습
predict(token_vocab, target_vocab, "별로에요")
    


0.99928445
POS 0


In [15]:
predict(token_vocab, target_vocab, "음식이 너무 맛있어요")
predict(token_vocab, target_vocab, "날씨가 더워서 아무것도 할 수 없었어")
predict(token_vocab, target_vocab, "경치가 예뻤으면 좋겠다!")

0.98873746
POS 0
1.0
NEG 1
0.9558535
POS 0
